# Puppet: Roles and profiles pattern
## Short guide to logic, design and implementation

## What was wrong with previous puppet repository

* A bit meshy and patchy
* A lot of dependencies, most unnecesary, implemented in a web on requires and stages
* Difficult to extend and rearrange components
* No separation of code and data

## A bit meshy and patchy

```puppet
class openstack_puppet::tweaks::owl_vlan_mapping( $odl_host ) {
  if $::osfamily == 'Debian' {
    $package = 'owl-vlan-mapping'
    ensure_resource('package', 'python-paramiko', {'ensure' => 'present'})
    ensure_resource('package', 'python-flask', {'ensure' => 'present'})
    ensure_resource('package', 'gunicorn', {'ensure' => 'present'})
    openstack_puppet::lib::custom_package {$package: }
  ...
```

## A lot of dependencies, most unnecesary, implemented in a web on requires and stages

```puppet
stage { 'repos': }
stage { 'rabbitmq': require => Stage['repos'] }
stage { 'mysql_server': require => Stage['repos'] }
stage { 'mysql_clients': require => Stage['mysql_server'] }
stage { 'keystone': require =>     [
  Stage['repos'],
  Stage['rabbitmq']
  ]
}
```

Difficult to extend and rearrange components

## No separation of code and data

Variables hardcoded or carried along from the top level class

```puppet
class openstack_puppet::single_node ( $network_tunnel_ip,
                                      $network_external_ip,
                                      $network_external_mask,
                                      $odl_tarball_url )
```
ODL tarball is not relevant to single_node class

## Roles and profiles pattern

What are profiles, what are roles and how they differ?

## Profile

A wrapper class that binds together variables, resources and classes in one functional unit describing a specific **Technology**. For example to deploy a Wordpress one needs to declare a user and group, install packages including apache and mysql and declare a vhost; this is a technology stack and should be declared as a profile

```puppet
class profiles::wordpress (
  $site_name,
  $wordpress_user_password,
  $mysql_root_password,
) {
  ## Create user
  group { 'wordpress':
    ensure => present,
    name   => $wordpress_group,
  }
  user { 'wordpress':
    ensure   => present,
    gid      => $wordpress_group,
    password => $wordpress_user_password,
    name     => $wordpress_group,
    home     => $wordpress_docroot,
  }

  ## Configure mysql
  class { 'mysql::server':
    root_password => $wordpress_root_password,
  }

  class { 'mysql::bindings':
    php_enable => true,
  }

  ## Configure apache
  include apache
  include apache::mod::php
  apache::vhost { $::fqdn:
    port    => $wordpress_port,
    docroot => $wordpress_docroot,
  }

  ## Configure wordpress
  class { '::wordpress':
    install_dir => $wordpress_docroot,
    db_name     => $wordpress_db_name,
    db_host     => $wordpress_db_host,
    db_password => $wordpress_db_password,
  }
}
```

## Role

A wrapper class that binds together specific technologies to create business units. Such a unit is described like "E-Shop server", "Mail server" etc.

```puppet
class roles::eshop {
  include profiles::wordpress
  include profiles::paypal
  include profiles::alphabankwebconnect
}
```

## Rules

* Roles only use incudes
* Roles only include profiles

* Profiles may declare resources and classes; use includes where sensible
* Profiles accept input

# What about data?

or

How do I pass data to profiles?

## Enter Hiera(rchical data)

Hiera is a Puppetlabs application that helps separate Puppet logic from data required to be passed to resources.

Hiera allows the user to define an hierarchy of (yaml) files in which to search for data when applying on a node. The information about the hierarchy is static in the configuration file:

```yaml
:backends:
  - yaml
:hierarchy:
  - "%{clientcert}"
  - "%{environment}"
  - "%{facter.os.name}"/"%{facter.os.release.full}"
  - common

:yaml:
  :datadir: /etc/puppet/environments/hieradata
```

The above hierarchy will look for data in files in the *datadir*. First it will look for data in file with the name of the node (clientcert), if this does not exist it will look for data in a file with the name of the environment used, then for a file with the OS version within a folder with the OS name and finally in the file named common.yaml.

## Hiera yaml files

Defining variables in yaml:
```yaml
profile::mysql::mysql::root_password: "root"
profile::mysql::mysql::bind_address: "0.0.0.0"
```
Corresponding class:
```puppet
class profile::mysql::mysql (
  $root_password,
  $bind_address,
){
  class { '::mysql::server':
    root_password    => $root_password,
    override_options => {
      'mysqld' => {
        'bind_address'           => $bind_address,
        'default_storage_engine' => 'InnoDB',
        'max_connections'        => 1024,
        'open_files_limit'       => -1
      }
    },
    restart          => true
  }
}
```

## Bringing it all together

* Profiles define technology stacks by declaring classes and resources
* Roles define physical objects by including profiles
* Hiera defines a lookup hierarchy for data
* Only data in hiera are to be changed for a new deployment

Note:

**Hiera should only define data for profiles, not for modules**

Correct:
```yaml
profile::mysql::mysql::root_password: "root"
```

Wrong:
```yaml
mysql::root_password: "root"
```

This helps avoid mistaken lookups; mysql module may be used by many profiles, one should be able to define the password at profile level as the module level may have higher priority.

## Other tools and integration

* R10k
* Foreman

## R10k

According to experts Roles&Profiles reside in the same repository as the Puppetfile. Same goes for Hiera files.

* Allows easier integration in an *environments* based topology
* Signifies that profiles and roles are specific to the installation, not to be shared as they don't necessarily make much sense in a different environment.

## Foreman

Foreman is great for roles and profiles pattern and can also work with Hiera. Different levels of integration with foreman can be achieved.

Profiles can be defined as *Config groups* and roles as *Hostgroups* if logic is to be stored in Foreman.
Also profiles and roles can be viewed as ordinary puppet classes to be assigned in hosts if logic is to be stored in puppet repo.

Profile variables can be defined as *Smart Class Parameters* if they are meant to be stored in Foreman.
Profile variables can be passed to Hiera with *Smart Variables* if the data are to be stored in hiera repo.

We opt for the second solution in both cases as it allows us to separate the ENC from actual Puppet and be able to use the puppet configuration for stand alone deployments dureng development and demos.

## More info

* [Hiera](https://docs.puppet.com/hiera/1/)
* [Roles and Profiles](http://garylarizza.com/blog/2014/02/17/puppet-workflow-part-2/)